# Mixed-Integer Linear Programming (MILP) for Local Interpretable Model-agnostic Explanations (LIME)

This study aims to formulate (and test) Local Interpretable Model-agnostic Explanations (LIME) using Mixed-Integer Linear Programming (MILP).

- Lucas Emanuel Resck Domingues
- Professor: Luciano Guimarães
- FGV-EMAp

## Introduction

The Optimization field is today one of the most important fields of Applied Mathematics.
Not only the theory and the research are very solid, but also the applications are everywhere, from Engineering to Management. In general, problems of this kind try to deal with the maximization or minimization of a function, given some conditions in the variables.

The main problem of Machine Learning, on the other hand, is to search for a function that represents the observed phenomenon using observed data. In fact, it can also be seen as an optimization problem, where the optimal function (given the data) is sought. For example, the goal of training a deep neural network is to search for a local mininum (with luck, a global mininum) for the loss function, given the data.

Machine Learning models have a problem: many of them are not interpretable. Some of them are so incognito that they are called "black-box", in reference to airplane's black-box.
This way, many researchers presented methods of explaining and interpreting Machine Learning model results.
One of them is Local Interpretable Model-agnostic Explanations (LIME), the focus of this study.

Suppose you have a text classification model (each text is assigned to a class) that says you if a text is mathematical or not. You want to understand why the model says this Introduction text (we call it $x$) is mathematical. LIME disturbs the text $x$ and verify the changes in probabilities $f(x)$. It tries to fit a simpler (but interpretable) model $g$, trying to predict the probabilities $f(x)$ over the pertubations of the input $x$. Mathematically, LIME tries to search for

$$\xi(x) = \mathrm{argmin}_{g \in G} \mathcal{L}(f, g, \pi_x) + \Omega(g),$$

being $G$ the set of desired interpretable functions, $\pi_x$ the locallity around $x$, $\mathcal{L}$ some measure of non proximity between $f, g$ in the locallity given by $\pi_x$, and $\Omega(g)$ being the complexity of $g$.
$\xi(x)$ will be a model that explains the prediction locally, but it will be interpretable too.

In general, LIME formulation is not MILP, but we can addapt the formulation to have it MILP.

## Setup

Only Python and Jupyter stuff. You can jump over this section.

In [1]:
from IPython.display import HTML
from pulp import LpVariable, LpProblem, value, LpStatus, LpMinimize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import string

In [2]:
%config Completer.use_jedi = False

## Model

In this section, we will fit a text classification model to predict if a **movie comment is positive or negative**.

### Data

The data we use is the famous **IMDB dataset**, a set of movie comments classified as **positive** or **negative**.

In [3]:
def preprocessing(text):
    '''Preprocess text.'''
    return text.replace('<br />', '')

In [10]:
df = pd.read_csv('../data/IMDB Dataset.csv')
df.review = df.review.apply(preprocessing)
print(df.iloc[0, 0][:200])
print()
print(df.iloc[0, 1][:100])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was it

positive


There is an example above of such a movie comment.

Now, we split the data between training and test for the model.

In [11]:
X = df.review.to_list()
y = df.sentiment.to_list()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### Classifier

We fit and evaluate our text classification model (classifier).

First, we need to convert the texts to vectors.
We do this using **TF-IDF**, a technique that create a vector for a text counting the number of times each word appears in the text (**TF**), but also normalizing by the number of times this word appears in all other texts (**IDF**).

In [13]:
tf_idf = TfidfVectorizer(
    strip_accents=None,
    lowercase=True,
    smooth_idf=True,
)
X_train = tf_idf.fit_transform(X_train)
X_train.shape

(40000, 94342)

As we can see, the dimensionality of these vectors is huge: 94342.
So we apply a **dimensionality reduction technique**, in our case Truncated SVD.

In [14]:
t_svd = TruncatedSVD(n_components=50, random_state=42)
X_train = t_svd.fit_transform(X_train)
X_train.shape

(40000, 50)

The vectors have their coordinates **"standardized"**, that is, they are transformed to have mean zero and variance 1.

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

The classification model used is **Support Vector Machines (SVM)**.
SVM tries to fit a hyperplane in the high-dimensional space where the vectors lie, trying to separate the vectors of different classes.

In [38]:
estimator = LinearSVC(
    dual=False,  # n_samples > n_features
    verbose=1,
    random_state=42
)

# We do a grid search to search for the best hyperparameters
svm = GridSearchCV(
    estimator,
    param_grid={'C': np.logspace(-2, 10, 13)},
    cv=5,
    n_jobs=-1,
    verbose=0
)

In [39]:
%%time
# Fit SVM
svm.fit(X_train, y_train)

# Fit probabilities
svm = CalibratedClassifierCV(svm.best_estimator_, cv=5)
svm.fit(X_train, y_train)

vector = Pipeline([
    ('tf_idf', tf_idf),
    ('t_svd', t_svd)
])

model = Pipeline([
    ('tf_idf', tf_idf),
    ('t_svd', t_svd),
    ('scaler', scaler),
    ('svm', svm)
])

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]CPU times: user 4.65 s, sys: 2.25 s, total: 6.9 s
Wall time: 7.96 s


We evaluate our model:

In [40]:
model.score(X_test, y_test)

0.8427

We see that SVM predicts 84\% of data right!

## Optimization

Now we enter the optimization problem.

### LIME

#### Original formulation

The formal original formulation of LIME is as follows.

Let $x \in \mathbb{R}^d$ be the **representation** of an instance (the vector of a text). We define $x' \in \{0, 1\}^{d'}$ the **interpretable representation** of $x$ (in our case, a binary vector indicating which words are the in text).
$G$ is a set of **interpretable models** $g \colon \{0, 1\}^{d'} \to \mathbb{R}$, and $\Omega(g)$ is defined as the complexity of a model $g \in G$.

Be $f \colon \mathbb{R}^d \to \mathbb{R}$ the model we want to explain, for example, the probability of a text $x$ being a negative movie review. $\pi_x(z)$ is some measure of locallity of $z$ around $x$ (it is greater if both vectors are closer, for example). $\mathcal{L}(f, g, \pi_x)$ is defined as the measure of non approximation between $f$ and $g$ in the locallity of $x$ given by $\pi_x$.

Given such definitions, the goal of LIME is to search for an interpretable model $g \in G$ such that

$$\xi(x) = \mathrm{argmin}_{g \in G} \mathcal{L}(f, g, \pi_x) + \Omega(g),$$

Both $\pi_x$ and $\mathcal{L}$ deal with locallity around $x$.
It is achieved disturbing the vector $x' \in \{0, 1\}^{d'}$, turning some coordinates into 0 (*i.e.*, erasing some words), resulting in various pertubations $z'$. From these interpretable representations we restore the representations $z$.

The very own LIME paper also sets the path for text classification models.
They define the set $G$ of interpretable models as the set of linear models $g(z') = w_g \cdot z'$.
The measure of non approximation is the locally weighted square loss

$$\mathcal{L}(f, g, \pi_x) = \sum_{z, z'} \pi_x(z) \left(f(z) - g(z')\right)^2$$

That is, the linear model $g(z')$ is trying to predict the probability $f(z)$.
They also define the complexity $\Omega$ as

$$\Omega(g) = \infty \cdot \mathbb{1}(\lVert w_g \rVert_0 > K),$$

that is, it is zero if $K$ or less components are not zero, and $\infty$ if more than $K$ components are not zero.

The locallity $\pi_x(z)$ is given by $\pi_x(z) = \exp\left(-D(x, y)^2/\sigma^2\right)$, with $D$ being the cosine distance between $x, y$, that is, $D(x, z) = \frac{1}{\pi} \arccos \frac{x \cdot z}{\lVert x \rVert \lVert z \rVert}$.

#### MILP formulation

The whole formulation above tries to search for a linear model over the perturbation of $x$ trying to predict the probabilities given by $x$.
The coefficients of the linear model will give us the importance of each word.

The idea is to have everything calculated, and to just optimize the coefficients of the linear model $g$.
This way, using the original formulation, we end up with an optimization problem that is not MILP.
See that the $g$ model in $\mathcal{L}$ is inside a square.
We also have a problem with $\Omega(g)$, that is intractable, according to the paper.
We will have to relax some of these.

A possible solution is as follows:

$$\begin{equation}
    \begin{aligned}
        &\underset{g \in G}{\text{Minimize}}&&\mathcal{L}(f, g, \pi_x) + \lambda \Omega(g)\\
        &\text{subject to} &&\mathcal{L}(f, g, \pi_x) = \sum_{z, z'} \pi_x(z) \lvert f(z) - g(z') \rvert,\\
        &&&\Omega(g) = |w_g|_1,\\
        &&&g(z') = w_g \cdot z', \ \ \forall z'\\
        &&&\mathcal{L} \in \mathbb{R},\\
        &&&\Omega \in \mathbb{R},\\
        &&&g(z') \in \mathbb{R}, \ \ \forall z'\\
        &&&w_g \in \{0, 1\}^{d'}\\
    \end{aligned}
\end{equation}$$

(Remember that $\pi_x(z)$, $f(z)$ and $z'$ are already calculated.)

We modify the formulation to have it MILP:

$$\begin{equation}
    \begin{aligned}
        &\underset{g \in G}{\text{Minimize}}&&\mathcal{L} + \lambda \Omega\\
        &\text{subject to} &&\mathcal{L} = \sum_{i} \pi_x(z_i) \varepsilon_i,\\
        &&&-\varepsilon_i \le f(z_i) - g_i \le \varepsilon_i, \ \ \forall i\\
        &&&\Omega = \sum_j \delta_j,\\
        &&&-\delta_j \le x_j \le \delta_j, \ \ \forall j\\
        &&&g_i = \sum_j z_{ij}' x_j, \ \ \forall i\\
        &&&\mathcal{L} \in \mathbb{R},\\
        &&&\Omega \in \mathbb{R},\\
        &&&g_i \in \mathbb{R}, \ \ \forall i\\
        &&&x_j \in \{0, 1\}, \ \ \forall j\\
        &&&\varepsilon_i \ge 0, \ \ \forall i\\
        &&&\delta_j \ge 0, \ \ \forall j\\
    \end{aligned}
\end{equation}$$

In fact, it is a weighted linear regression with LASSO regularization.

### Calculation of parameters

In [ ]:
def f(text):
    '''Probability of a text'''
    return model.predict_proba([text])[0]

In [ ]:
def pi(x, z, sigma_2=-1/np.log(0.1)):
    '''Weights of locallity.'''
    x = vector.transform([x])[0]
    z = vector.transform([z])[0]
    # If null vector
    if np.abs(z).sum() == 0:
        return 0
    # Cosine of angle between vectors
    cos = np.dot(x, z)/(np.linalg.norm(x)*np.linalg.norm(z))
    # If cosine is like 1.00008
    if cos > 1:
        cos = 1
    # Angle between vectors, normalized to between 0 and 1
    D = np.arccos(cos)*2/np.pi
    return np.exp(-D**2/sigma_2)

In [ ]:
def parameters(split, which_class, M, N, K):
    '''Return the parameters for LIME optimization.'''
    # Perturbations
    z_line = []
    # Probabilities
    f_z = []
    # Weights
    pi_x = []
    
    for i in range(N):
        # Choose a random number of words to remove, between 1 and (split - 1)
        n = np.random.choice(range(1, M))
        # Remove n random words
        indices = np.random.choice(range(M), size=n, replace=False)
        
        # The pertubartion
        perturbation = np.ones(M)
        for index in indices:
            perturbation[index] = 0
        z_line.append(perturbation)
            
        # The probability and weight
        text = ' '.join([word for (j, word) in enumerate(split) if perturbation[j]])
        f_z.append(f(text)[which_class])
        pi_x.append(pi(example, text))
    
    return z_line, f_z, pi_x

### Linear optimization

In [ ]:
def optimization(z_line, f_z, pi_x, M, N, K, lambda_=0.5):
    '''The MILP for LIME.'''
    prob = LpProblem("LIME", LpMinimize)
    
    # Variables
    L = LpVariable('L')
    Omega = LpVariable('Omega')
    epsilon = [LpVariable('epsilon_{}'.format(i)) for i in range(N)]
    delta = [LpVariable('delta_{}'.format(i)) for i in range(M)]
    g = [LpVariable("g(z'_{})".format(i)) for i in range(N)]
    x = [LpVariable('x_{}'.format(j)) for j in range(M)]
#     y = [LpVariable('y_{}'.format(j), 0, 1, cat='Integer') for j in range(M)]
    
    # Objective
    prob += L + lambda_*Omega
    
    # Constraints
    prob += L == sum([pi_x[i]*epsilon[i] for i in range(N)])
    prob += Omega == sum(delta)

    for i in range(N):
        prob += -epsilon[i] <= f_z[i] - g[i]
        prob += epsilon[i] >= f_z[i] - g[i]
        prob += g[i] == sum([z_line[i][j]*x[j] for j in range(M)])

    infinity = 100000
    for j in range(M):
        prob += -delta[j] <= x[j]
        prob += delta[j] >= x[j]
        
#         prob += -infinity*y[j] <= x[j]
#         prob += infinity*y[j] >= x[j]

#     prob += sum(y) <= K

    print('Solving MILP...')
    status = prob.solve()
    print('Done.')
    
    return prob, status, x

In [ ]:
def visualize(split, importances):
    '''Visualize the importance of each word in the classification.'''
    max_abs_importance = np.max(np.abs(importances))
    # Green
    positive = np.array([0, 255, 0])
    white = np.array([255, 255, 255])
    # Red
    negative = np.array([255, 0, 0])
    spans = []
    for i, word in enumerate(split):
        if importances[i] >= 0:
            color = white + (positive - white)/max_abs_importance*importances[i]
        else:
            color = white + (negative - white)/((-1)*max_abs_importance)*importances[i]
        spans.append(
            '<span style="background-color: RGB({R}, {G}, {B})">{word}</span>'.format(
                word=word,
                R=color[0],
                G=color[1],
                B=color[2]
            )
        )
            
    html = ' '.join(spans)
    return HTML(html)

In [ ]:
def lime(text, which_class, N=None, K=None):
    # Split 
    split = text.split()
    M = len(split)
    if N is None:
        N = 5*M
#     if K is None:
#         K = min([M, 20])
                
    z_line, f_z, pi_x = parameters(split, which_class, M, N, K)
        
    prob, status, x = optimization(z_line, f_z, pi_x, M, N, K)
    
    importances = [value(i) for i in x]    
    print(dict(zip(split, importances)))    
    
    return visualize(split, importances)

### Examples

In [ ]:
example = 'This movie is awful, I regret seing it, it is a bad movie.'
example

In [ ]:
model.predict([example])

In [ ]:
model.predict_proba([example])

In [ ]:
lime(example, 0)

## References

- https://arxiv.org/pdf/1602.04938.pdf
- https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
- https://vanderbei.princeton.edu/tex/talks/MOPTA14/L1_reg.pdf